In [ ]:
import numpy as np
import cv2
import os
import torch
import spacy
import pandas as pd
import torchvision.transforms as T

from torchvision.io.video import read_video
from tqdm.notebook import tqdm
from PIL import Image
from torch.utils.data import DataLoader,Dataset
from collections import Counter
from IPython.display import clear_output

In [ ]:
!python -m spacy download ru_core_news_lg

2023-09-30 05:14:22.157004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 84.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ./drive/MyDrive/rutube_hackathon_novosibirsk.zip ./rutube_hackathon_novosibirsk.zip

In [ ]:
!zip -FF ./rutube_hackathon_novosibirsk.zip --out ./pleasework.zip
clear_output()

In [ ]:
!unzip -qq ./pleasework.zip

In [ ]:
spacy_ru = spacy.load("ru_core_news_lg")

text = "Сергей Чуприн любит hi! ОН - КАРТОфЕЛЕЛЮБ!!!"
[token.text.lower() for token in spacy_ru.tokenizer(text)]

['сергей',
 'чуприн',
 'любит',
 'hi',
 '!',
 'он',
 '-',
 'картофелелюб',
 '!',
 '!',
 '!']

In [ ]:
train_csv = pd.read_csv("./rutube_hackathon_novosibirsk/train/train.csv")

In [ ]:
train_csv["len"] = train_csv["description"].apply(lambda x: len(x.split()))

In [ ]:
train_csv.len.max()

321

In [ ]:
class Vocabulary:
    def __init__(self,freq_threshold):
        self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}

        self.stoi = {v:k for k,v in self.itos.items()}

        self.freq_threshold = freq_threshold

    def __len__(self): return len(self.itos)

    @staticmethod
    def tokenize(text):
        return [token.text.lower() for token in spacy_ru.tokenizer(text)]

    def build_vocab(self, sentence_list):
        frequencies = Counter()
        idx = 4

        for sentence in tqdm(sentence_list):
            for word in self.tokenize(sentence):
                frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self,text):
        tokenized_text = self.tokenize(text)
        return [ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text ]

In [ ]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions, video_files, transform=None,freq_threshold=1):
        self.root_dir = root_dir
        self.transform = transform

        self.imgs = video_files
        self.captions = captions

        # for file in tqdm(captions_files):
        #     with open(self.root_dir + "/train_stt/" + file) as cf:
        #             self.captions.append(" ".join(cf.readlines()))

        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocab(self.captions)


    def __len__(self):
        return len(self.imgs)

    def _read_video(self, path, frames_num=25, window=30):
        frames = []
        cap = cv2.VideoCapture(self.root_dir + "/train_video/" + path)

        fps = int(cap.get(cv2.CAP_PROP_FPS))

        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        N = length // (frames_num)

        current_frame = 1
        for i in range(length):
            ret, frame = cap.read(current_frame)
            if ret and i == current_frame and len(frames) < frames_num:
                size = 226, 226
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, size)
                frames.append(frame)
                current_frame += N


        cap.release()

        return np.array(frames)

    def __getitem__(self,idx):
        caption = self.captions[idx]
        img = self.imgs[idx]
        img = np.rollaxis(self._read_video(self.imgs[idx]), 3, 0)
        img = np.array(np.array(img) / 255, dtype=np.float32)


        caption_vec = []
        caption_vec += [self.vocab.stoi["<SOS>"]]
        caption_vec += self.vocab.numericalize(caption)
        caption_vec += [self.vocab.stoi["<EOS>"]]
        if len(caption_vec) < 300:
            for i in range(300 - len(caption_vec)):
                caption_vec.append(0)

        return torch.tensor(img), torch.tensor(caption_vec[:300])

In [ ]:

data_location =  "./rutube_hackathon_novosibirsk/train"
BATCH_SIZE = 10

transforms = T.Compose([
    T.Resize(226),
    T.RandomCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])


dataset =  FlickrDataset(
    root_dir=data_location,
    captions=train_csv.description.tolist(),
    video_files=train_csv.video_name,
    transform=transforms
)

data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # batch_first=False
)

vocab_size = len(dataset.vocab)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

  0%|          | 0/500 [00:00<?, ?it/s]

device(type='cuda', index=0)

### 3) Defining the Model Architecture

Model is seq2seq model. In the **encoder** pretrained ResNet model is used to extract the features. Decoder, is the implementation of the Bahdanau Attention Decoder. In the decoder model **LSTM cell**.

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.models as models
import torchvision.models.video as video

from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        self.swin_t = video.swin3d_s(pretrained=True)
        # self.resnet = models.resnet50(pretrained=True)
        for param in self.swin_t.parameters():
            param.requires_grad_(False)
        # transforms = video.swin_transformer.SwinTransformer3d()
        modules = list(self.swin_t.children())[:-2]
        self.swin_t = nn.Sequential(*modules)
        # print(len(self.swin_t.))


    def forward(self, images):
        # print(images.shape)
        features = self.swin_t(images)                                 #(batch_size,2048,7,7)
        # print(features.shape)
        # features = features.permute(0, 2, 3, 4, 1)                           #(batch_size,7,7,2048)
        features = features.view(features.size(0), -1, features.size(-1)) #(batch_size,49,2048)
        # features = features.view(-1) #(batch_size,49,2048)
        # print(features.shape)

        return features


In [ ]:
class Attention(nn.Module):
    def __init__(self, encoder_dim,decoder_dim,attention_dim):
        super(Attention, self).__init__()

        self.attention_dim = attention_dim

        self.W = nn.Linear(decoder_dim,attention_dim)
        self.U = nn.Linear(encoder_dim,attention_dim)

        self.A = nn.Linear(attention_dim,1)




    def forward(self, features, hidden_state):
        u_hs = self.U(features)
        w_ah = self.W(hidden_state)

        combined_states = torch.tanh(u_hs + w_ah.unsqueeze(1))

        attention_scores = self.A(combined_states)
        attention_scores = attention_scores.squeeze(2)


        alpha = F.softmax(attention_scores,dim=1)

        attention_weights = features * alpha.unsqueeze(2)
        attention_weights = attention_weights.sum(dim=1)

        return alpha,attention_weights


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self,embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim,drop_prob=0.3):
        super().__init__()

        self.vocab_size = vocab_size
        self.attention_dim = attention_dim
        self.decoder_dim = decoder_dim

        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.attention = Attention(encoder_dim,decoder_dim,attention_dim)


        self.init_h = nn.Linear(encoder_dim, decoder_dim)
        self.init_c = nn.Linear(encoder_dim, decoder_dim)
        self.lstm_cell = nn.LSTMCell(embed_size+encoder_dim,decoder_dim,bias=True)
        self.f_beta = nn.Linear(decoder_dim, encoder_dim)


        self.fcn = nn.Linear(decoder_dim,vocab_size)
        self.drop = nn.Dropout(drop_prob)



    def forward(self, features, captions):
        embeds = self.embedding(captions)

        h, c = self.init_hidden_state(features)

        seq_length = len(captions[0])-1
        batch_size = captions.size(0)
        num_features = features.size(1)

        preds = torch.zeros(batch_size, seq_length, self.vocab_size).to(device)
        alphas = torch.zeros(batch_size, seq_length,num_features).to(device)

        for s in range(seq_length):
            alpha,context = self.attention(features, h)
            lstm_input = torch.cat((embeds[:, s], context), dim=1)
            h, c = self.lstm_cell(lstm_input, (h, c))

            output = self.fcn(self.drop(h))

            preds[:,s] = output
            alphas[:,s] = alpha


        return preds, alphas

    def generate_caption(self,features, max_len=300 ,vocab=None):

        batch_size = features.size(0)
        h, c = self.init_hidden_state(features)

        alphas = []
        word = torch.tensor(vocab.stoi['<SOS>']).view(1,-1).to(device)
        embeds = self.embedding(word)


        captions = []

        for i in range(max_len):
            alpha,context = self.attention(features, h)

            alphas.append(alpha.cpu().detach().numpy())

            lstm_input = torch.cat((embeds[:, 0], context), dim=1)
            h, c = self.lstm_cell(lstm_input, (h, c))
            output = self.fcn(self.drop(h))
            output = output.view(batch_size,-1)


            predicted_word_idx = output.argmax(dim=1)

            captions.append(predicted_word_idx.item())

            if vocab.itos[predicted_word_idx.item()] == "<EOS>":
                break

            embeds = self.embedding(predicted_word_idx.unsqueeze(0))

        return [vocab.itos[idx] for idx in captions],alphas


    def init_hidden_state(self, encoder_out):
        mean_encoder_out = encoder_out.mean(dim=1)
        h = self.init_h(mean_encoder_out)
        c = self.init_c(mean_encoder_out)
        return h, c


In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self,embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim,drop_prob=0.3):
        super().__init__()
        self.encoder = EncoderCNN()
        self.decoder = DecoderRNN(
            embed_size=embed_size,
            vocab_size = len(dataset.vocab),
            attention_dim=attention_dim,
            encoder_dim=encoder_dim,
            decoder_dim=decoder_dim
        )

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs


### 4) Setting Hypperparameter and Init the model

In [37]:
#Hyperparams
embed_size=300
vocab_size = len(dataset.vocab)
attention_dim=256
encoder_dim=768
decoder_dim=512
learning_rate = 3e-4


In [ ]:
#init model
model = EncoderDecoder(
    embed_size=embed_size,
    vocab_size=vocab_size,
    attention_dim=attention_dim,
    encoder_dim=encoder_dim,
    decoder_dim=decoder_dim
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Swin3D_S_Weights.KINETICS400_V1`. You can also use `weights=Swin3D_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# !pip install av

In [ ]:
#helper function to save the model
def save_model(model,num_epochs):
    model_state = {
        'num_epochs':num_epochs,
        'embed_size':embed_size,
        'vocab_size':len(dataset.vocab),
        'attention_dim':attention_dim,
        'encoder_dim':encoder_dim,
        'decoder_dim':decoder_dim,
        'state_dict':model.state_dict()
    }

    torch.save(model_state,'./drive/MyDrive/attention_model_state.pth')

## 5) Training Job from above configs

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import nltk
from nltk.translate import meteor
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def score(text, text_sum):
    text = text.replace("<PAD>", "")
    if isinstance(text_sum, str):
        return round(meteor([word_tokenize(text)],word_tokenize(text_sum)), 4)
    else:
        return 0

In [36]:
num_epochs = 25
print_every = 10

for epoch in tqdm(range(1, num_epochs + 1)):
    losses = []
    print(f"Epoch: {epoch}")
    try:
        for idx, (image, captions) in tqdm(enumerate(data_loader), total=len(data_loader)):
            # print(image.shape[2])
            # if image.shape[2] < 50:
            #     continue
            image, captions = image.to(device),captions.to(device)

            optimizer.zero_grad()

            outputs,attentions = model(image, captions)

            loss = criterion(outputs.view(-1, vocab_size), captions[:, 1:].reshape(-1))

            loss.backward()

            optimizer.step()
            losses.append(loss.item())

            if idx % print_every == 0:
                model.eval()
                with torch.no_grad():
                    dataiter = iter(data_loader)
                    img,targ = next(dataiter)
                    features = model.encoder(img[0:1].to(device))
                    caps, alphas = model.decoder.generate_caption(features, vocab=dataset.vocab)
                    caption = ' '.join(caps)
                    target = " ".join([dataset.vocab.itos[i] for i in targ[0:1][0][1:].tolist()])
                    print("true:", target)
                    print("pred:", caption)
                    print("loss:", loss.item(), "score:", score(target, caption))


                model.train()
    except Exception as ex:
        print("Err:", ex)
        raise ex
    if len(losses) != 0:
        print("Epoch loss: {:.5f}".format(sum(losses) / len(losses)))
    else:
        print(losses)
    #save the latest model
    save_model(model,epoch)
    # clear_output(wait=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 1


  0%|          | 0/50 [00:00<?, ?it/s]

true: « наше дело » представляет реалити - шоу " путь бойца " . в этом выпуске 20 бойцов сразятся за право быть лучшим , чтобы стать частью команды « наше дело » и получить контракт на 5 миллиона рублей . участников ждут бескомпромиссные драки , испытания на прочность , скорость и ловкость ! кто справится с тяжелейшей полосой препятствий в сложных погодных условиях ? кто получит травму во время испытаний и не сможет продолжить свое участие в шоу ? кому такой путь окажется не по зубам ? все это вы узнаете в первом выпуске реалити - шоу « путь бойца » . по вопросам рекламы пишите на email <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

  0%|          | 0/50 [00:00<?, ?it/s]

...

true: максим константинович и ольга пдн примеряют на себя роль автогонщиков и участвуют в захватывающей погоне за мусоровозом , чтобы спасти фоторобота . позже наш начальник выходит на футбольное поле , а вечером пытается пройти в ночной клуб , чтобы контролировать нашу молодежь во всех сферах жизни , а потом помогает олегу - фотороботу , достать лампочку , которую они с лёхой - подмогой своровали довольно странным способом ! поддержите видео лайком , а канал подпиской ! <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

  0%|          | 0/50 [00:00<?, ?it/s]

true: даня милохин и юля гаврилина больше не вместе , валю карнавал обвиняют в читерстве , а инстаграм поставил под угрозу заработок блогеров ! все самые актуальные новости в новом выпуске tiktok дайджеста . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

  0%|          | 0/50 [00:00<?, ?it/s]

true: какая скульптура снова входит и моду и за чем охотятся коллекционеры ? можно ли стать известным скульптором , не имея госзаказов и большой мастерской ? как платформы современного искусства помогают молодым скульпторам найти свой путь к славе . об этом и еще о том , как иван серый лепит с закрытыми глазами русалку прямо в студии расскажет артмеханика . пропустишь – значит пропустишь еще и редкое выступление романтичных даша и сережа . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

In [38]:
checkpoint = torch.load("drive/MyDrive/attention_model_state.pth")

In [39]:
#Hyperparams
embed_size=300
vocab_size = len(dataset.vocab)
attention_dim=256
encoder_dim=768
decoder_dim=512
learning_rate = 3e-4


In [40]:
model = EncoderDecoder(
    embed_size=embed_size,
    vocab_size =vocab_size,
    attention_dim=attention_dim,
    encoder_dim=encoder_dim,
    decoder_dim=decoder_dim
).to(device)
model.load_state_dict(checkpoint["state_dict"])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Swin3D_S_Weights.KINETICS400_V1`. You can also use `weights=Swin3D_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [41]:

data_location =  "./rutube_hackathon_novosibirsk/train"
BATCH_SIZE = 1

transforms = T.Compose([
    T.Resize(226),
    T.RandomCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])


dataset =  FlickrDataset(
    root_dir=data_location,
    captions=train_csv.description.tolist(),
    video_files=train_csv.video_name,
    transform=transforms
)

data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # batch_first=False
)

vocab_size = len(dataset.vocab)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

  0%|          | 0/500 [00:00<?, ?it/s]

device(type='cuda', index=0)

In [42]:
scores = []
model.eval()

for idx, (image, captions) in tqdm(enumerate(data_loader), total=len(data_loader)):
    image, captions = image.to(device),captions.to(device)
    with torch.no_grad():
        dataiter = iter(data_loader)
        img,targ = next(dataiter)
        features = model.encoder(img[0:1].to(device))
        caps, alphas = model.decoder.generate_caption(features, vocab=dataset.vocab)
        caption = ' '.join(caps)
        target = " ".join([dataset.vocab.itos[i] for i in targ[0:1][0][1:].tolist()])
        scores.append(score(target, caption))
        # print("true:", target)
        # print("pred:", caption)
        # print("loss:", loss.item(), "score:", score(target, caption))

print(sum(scores) / len(scores))


  0%|          | 0/500 [00:00<?, ?it/s]

0.42888460000000067
